In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict

base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함



## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)
custom_train_test_split.__doc__

train = df[df.kind=='train']
train, valid = custom_train_test_split(train, ratio=0.7) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

'df를 입력받아서, userid들을 랜덤해서 1-ratio만큼의 user만 선별하고, 마지막 제출껀만 만들어서 train, test로 리턴 '

((1845634, 23), (1964, 23), (260114, 23))

((1845634, 22), (1845634, 1), (1964, 22), (1964, 1))

In [5]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std', 'tag_std',
         'tag_mean',
         'tag_sum',
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
    "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
    "metric": "auc",
}

model = lgb.LGBMClassifier(
    **params,
    n_estimators=10,
    silent=-1,
)

In [6]:
model.fit(
    X=x_train[FEATS],
    y=y_train,
    early_stopping_rounds=100,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="roc_auc",
    verbose=5,
)

preds = model.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[5]	train's auc: 0.733106	valid's auc: 0.676377
[10]	train's auc: 0.73597	valid's auc: 0.678224


LGBMClassifier(learning_rate=0.01, metric='auc', n_estimators=10,
               objective='binary', seed=42, silent=-1)

In [9]:
lgbm_predict(test, model, FEATS, 'test.csv')

writing prediction : output/test.csv
